In [2]:
import os
import numpy as np
import matplotlib.pyplot as plt
%matplotlib ipympl
plt.rcParams['figure.figsize'] = [7, 6]

import lsst.daf.persistence as dafPersist
import lsst.afw.display as afwDisplay
import lsst.afw.display.utils as afwDisplayUtils
import lsst.afw.geom as afwGeom
import lsst.afw.math as afwMath
import lsst.afw.cameraGeom as cameraGeom
import lsst.afw.cameraGeom.utils as cgUtils

In [3]:
afwDisplay.setDefaultBackend("matplotlib")

In [4]:
if True:
    if True:
        from lsst.obs.lsst.phosim import PhosimMapper as camMapper
    else:
        from lsst.obs.lsst import LsstCamMapper as camMapper

    dataId = dict(visit=204595, raftName='R11', detectorName='S11')
else:
    from lsst.obs.lsst.ts8 import Ts8Mapper as camMapper
    dataId = dict(raftName='RTM-010', detectorName='S11', visit=201807241028453)
    
camera = camMapper._makeCamera()

#### Get a butler so we can read some images

Only needed for this notebook

In [5]:
if False:
    butler = dafPersist.Butler(os.path.join("/project/rhl/Data/testdata_obs_lsst", "Outputs", cameraName, "rerun/rlupton", "tmp"))
else:
    butler = None

## Routines to read per-amplifier images and convert coordinates to/from CCD pixel coordinates

In [6]:
from lsst.obs.lsst.cameraTransforms import LsstCameraTransforms, channelToAmp, getAmpImage

### Setup some DM stuff that the butler would usually handle

Note that we need a camera to initialise the LsstCameraTransforms


In [7]:
raftName, detectorName = dataId["raftName"], dataId["detectorName"]
detectorName = "%s_%s" % (raftName, detectorName)

lct = LsstCameraTransforms(camera)

#### Get a DM detector and ask it about itself

In [8]:
det = lct.getDetector(detectorName)
print(det.getName(), det.getType(), det.getPhysicalType(), det.getSerial())

R11_S11 DetectorType.SCIENCE ITL ITL-3800C-017-Dev


#### Convert some CCD coordinates to amp coordinates

Validate this by looking at some images with recognisable pixels

In [13]:
# Choose some pixels to check by eye
if camera.getName() == "ts8":
    if False:
        cx, cy = 1543, 2022   # C03
    elif False:
        cx, cy = 2942, 2035   # C05
    else:
        cx, cy = 2682, 1969   # C15
elif camera.getName() in ("lsstCam", "phosim",):
    if False:
        cx, cy = 3805, 3708   # C07
    elif True:
        cx, cy = 3332, 1147   # C16
else:
    raise NameError(camera.getName())

#
# Convert from the CCD coordinates to per-amp coordinates
#
channel, ampX, ampY = lct.ccdPixelToAmpPixel(cx, cy, detectorName)

if butler is not None:
    #
    # Read the raw amp data, assuming that the butler isn't adding any information
    #
    rawa = getAmpImage(butler, dataId, channel)

    #
    # Display the amp image, showing the selected point
    #
    disp = afwDisplay.Display(4, reopenPlot=True)

    stats = afwMath.makeStatistics(rawa.image, afwMath.MEDIAN | afwMath.STDEVCLIP)
    med = stats.getValue(afwMath.MEDIAN)
    std = stats.getValue(afwMath.STDEVCLIP)
    disp.scale('linear', med - 1*std, med + 2*std)

    disp.mtv(rawa, title=channelToAmp(lct.getDetector(detectorName), channel).getName())
    disp.dot('+', ampX, ampY, ctype=afwDisplay.RED)
    disp.zoom(16, ampX, ampY)

### Plot the whole focal plane

In [16]:
plt.close('all')
cgUtils.plotFocalPlane(camera)
plt.title(camera.getName());

FigureCanvasNbAgg()

#### Convert a CCD position to focal plane coordinates and plot it

In [17]:
detectorName = "%(raftName)s_%(detectorName)s" % (dataId)

for ccdXY, color in [
    ((-0.5, -0.5), "red"),   # bottom left corner of CCD
    (camera[detectorName].getBBox().getEnd() - afwGeom.ExtentD(0.501, 0.501), "yellow"),  # top right of CCD
    ]:
    fpXMm, fpYMm = lct.ccdPixelToFocalMm(*ccdXY, detectorName)

    plt.plot(fpXMm, fpYMm, '+', color=color)
    print("%s (%8.3f, %8.3f)pix -> (%8.3f, %8.3f)mm" % (detectorName, ccdXY[0], ccdXY[1], fpXMm, fpYMm))

R11_S11 (  -0.500,   -0.500)pix -> (-141.886, -147.043)mm
R11_S11 (4071.499, 3999.499)pix -> (-108.730, -100.580)mm


#### Demonstrate the reverse mapping;  focal plane back to CCD

In [18]:
dname, ccdX, ccdY = lct.focalMmToCcdPixel(fpXMm, fpYMm)
print("(%.3f, %.3f)mm -> %s (%.3f, %.3f)pix" % (fpXMm, fpYMm, dname, ccdX, ccdY))

(-108.730, -100.580)mm -> R11_S11 (4071.499, 3999.499)pix


#### And focal plane to amplifier

In [19]:
dname, channel, ampX, ampY = lct.focalMmToAmpPixel(fpXMm, fpYMm)
print("(%.3f, %.3f)mm -> %s channel %d (%.3f, %.3f)pix" % (fpXMm, fpYMm, dname, channel, ampX, ampY))

(-108.730, -100.580)mm -> R11_S11 channel 9 (3.000, 0.000)pix


In [20]:
if butler is not None:
    #
    # Read the raw amp data, assuming that the butler isn't adding any information
    #
    rawa = getAmpImage(butler, dataId, channel)

    #
    # Display the amp image, showing the selected point
    #
    disp = afwDisplay.Display(5, reopenPlot=True)

    disp.scale('asinh', 'zscale', Q=2)

    disp.mtv(rawa, title=channelToAmp(lct.getDetector(detectorName), channel).getName())
    disp.dot('+', ampX, ampY, ctype=afwDisplay.RED)
    disp.zoom(32, ampX, ampY)